In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import date, timedelta

In [2]:
def WebNovels_RawData(driver_path, start_date, end_date):
    #Web scraping with selenium
    start_date = date.fromisoformat(start_date)
    end_date = date.fromisoformat(end_date)
    raw_data = []
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    delta = end_date - start_date
    for i in range(delta.days + 1):
        publish_date = start_date + timedelta(days=i)
        url = ('https://yomou.syosetu.com/search.php?word=&notword=&genre=&type=re&mintime=&maxtime=&minlen=&maxlen=&min_globalpoint=&max_globalpoint=&minlastup=&maxlastup=&minfirstup=' 
            + str(publish_date.year) + '%2F' + str(publish_date.month) + '%2F' + str(publish_date.day) + '&maxfirstup=' + str(publish_date.year) + '%2F' + str(publish_date.month) + '%2F' + str(publish_date.day) + '&order=favnovelcnt')
        driver.get(url)
        retries = 0
        while retries < 2:
            try:    
                results = driver.find_elements(By.CLASS_NAME,'searchkekka_box')
                for i in results:
                    temp = i.text
                    temp = temp + '\n' + str(publish_date)
                    raw_data.append(temp.split('\n'))
                while True:
                    try:
                        driver.find_element(By.CLASS_NAME,'nextlink').click()
                        break
                    except:
                        retries = 3
                        break
            except TimeoutException:
                print('Timeout', publish_date)
                driver.refresh()
                retries += 1
    driver.quit()
    return raw_data  

In [3]:
def WebNovels_DataCleaning(raw_data):
    df = pd.DataFrame(raw_data)
    df = df.drop(df.columns[[2,3,11]], axis=1)
    
    df[9] = df[9].replace('/','-',regex=True)
    for i in list(df):
        df[i] =df[i].replace(',', '',regex=True)
        df[i] =df[i].replace('／', '／',regex=True)
    df[['Author','ID']] = df[1].str.split('／小説情報／',expand=True)
    df[['Sub_Genre','Genre']] = df[7].str.split('〔',expand=True)
    df[['Latest_Update','Blank']] = df[9].str.split(' ',expand=True)
    df[['Blank','Word_Count']] = df[9].str.split('（',expand=True)
    df[['Weekly_Viewers','Reviews']] = df[10].str.split('レビュー数',expand=True)
    df[['Bookmarks','Ratings','Total_Scores']] = df[12].str.split('人',expand=True)
    
    df['Keywords'] = df[8].replace('キーワード：','',regex=True)
    df['Author'] = df['Author'].replace('作者：','',regex=True)
    df['ID'] = df['ID'].replace('Nコード：','',regex=True)
    df['Latest_Update'] = df['Latest_Update'].replace('最終更新日：','',regex=True)
    df['Genre'] = df['Genre'].replace('〕','',regex=True)
    df['Sub_Genre'] = df['Sub_Genre'].replace('ジャンル：','',regex=True)
    
    df = df.drop(columns=[1,7,8,9,10,12,'Blank'])
    df = df.rename(columns={0: 'Title', 4: 'Status', 5: 'Parts', 6: 'Abstract', 13: 'Publish Date'})
    for i in ('Word_Count','Weekly_Viewers','Reviews','Bookmarks','Ratings','Total_Scores', 'Parts'):
        df[i] = df[i].str.extract('(\d+)')
        df[i] = pd.to_numeric(df[i])

    df['Title_Length'] = df['Title'].str.len()
    df['Abstract_Length'] = df['Abstract'].str.len()
    df['Average_Score'] = df['Total_Scores']/df['Ratings']
    df['Average_Score'] = df['Average_Score'].round(4)
    
    #Translate strings from Japanese to English
    df = df.replace({
    'Status': {'連載中': 'On Going','完結済':'Completed'},
    'Genre': {'恋愛': 'Romance', 'ファンタジー': 'Fantasy','文芸':'Literature',
              'SF':'Sci-fi','その他':'Others','ノンジャンル':'Non-genre'},
    'Sub_Genre': {'異世界': 'Fantasy World','現実世界':'Real World','ハイファンタジー':'High Fantasy',
                  'ローファンタジー':'Low Fantasy','純文学':'Pure Literature','ヒューマンドラマ':'Drama',
                  '歴史':'Historical Fiction','推理':'Mystery','ホラー':'Horror','アクション':'Action Fiction',
                  'コメディー':'Comedy','VRゲーム':'Virtual Reality','宇宙':'Space','空想科学':'Science Fantasy',
                  'パニック':'Disaster Fiction','童話':'Fairy Tale','詩':'Poetry','エッセイ':'Essay','リプレイ':'TRPG',
                  'その他':'Others','ノンジャンル':'Non-genre'}})
    
    return df

In [4]:
driver_path = 'C:/Users/eva01/Desktop/Python/chromedriver.exe'
raw = WebNovels_RawData(driver_path, '2007-01-01', '2020-12-31')

In [21]:
data = WebNovels_DataCleaning(raw)
print('The number of observation is',len(data.index))

The number of observation is 400373


In [22]:
data.head()

,Title,Status,Parts,Abstract,Publish Date,Author,ID,Sub_Genre,Genre,Latest_Update,Word_Count,Weekly_Viewers,Reviews,Bookmarks,Ratings,Total_Scores,Keywords,Title_Length,Abstract_Length,Average_Score
0,満月の騎士,Completed,25,どこかで聞いたことのあるような名前の主人公はある時、幽霊の憑いた木刀を手にいれる。さて、これ...,2007-01-01,雨月,N3460B,Non-genre,Non-genre,2007-04-29,82039,100,0,3,2.0,12.0,霊 シリアス コメディ ハッピーエンド らぶらぶ 高校生 現代(モダン),5,76,6.0
1,エイト・アイズ,On Going,24,狐宮龍弥は天才である。天才であるが故に子供のころから色々と危険な目にもあった。なので、護身術...,2007-01-01,サカガミ ヤスヒロ,N3458B,Non-genre,Non-genre,2007-07-02,47311,100,0,0,0.0,0.0,目 ８ 悪霊 霊 妖怪 高校生 現代(モダン) 霊界/地獄/天国 残酷な描写有り,7,106,NaN
2,フェア,On Going,33,自分の村を失った少年や大人が、教会で新たな仲間と出会い、世界を守るために闇と戦っていく…悲し...,2007-01-01,藍,N3456B,Non-genre,Non-genre,2007-11-09,94887,100,0,0,0.0,0.0,戦争 ファンタジー 少年 十字架 双子,3,65,NaN
3,SPhinX,On Going,7,SPhinxとは、SP(SecurityPolice：要人警護、警護官)とスフィンクス（伝説...,2007-01-02,瑛彪・玄彪,N3483B,High Fantasy,Fantasy,2007-12-28,9244,100,0,0,1.0,2.0,ＳＰ 中学生 高校生 太古 現代(モダン),6,82,2.0
4,フェチ性人格障害,On Going,24,誰でも潜む欲求。普通に生活し普通に生きてきたはずの男性や女性のふとした事からのフェチシズム。...,2007-01-02,かずん。,N3485B,Non-genre,Non-genre,2007-01-28,27414,100,0,0,0.0,0.0,R15 精神病 解離 人格障害 フェチシズム,8,93,NaN


In [23]:
data.to_excel('webnoveldata.xlsx',index=False,encoding='cp1252')